In [ ]:
import pandas as pd
import geopandas as gpd
from sklearn.preprocessing import MinMaxScaler

def read_data(filename, city):
    df = pd.read_excel(filename)
    return df[df["市"] == city]

def calculate_scores(df, columns):
    scaler = MinMaxScaler()
    for col in columns:
        new_col = col + "_score"
        df[new_col] = scaler.fit_transform(df[[col]]) * 100
    return df

def extract_coordinates(df):
    df["lon"] = df["栅格编号"].str.split("-", expand=True)[1].astype(float)
    df["lat"] = df["栅格编号"].str.split("-", expand=True)[2].astype(float)
    return df

def create_geodataframe(df):
    return gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df["lon"], df["lat"]))

def score_grid(df, bounds, tbd):
    p = tbd.area_to_params(bounds, accuracy=5000)
    grid, p = tbd.area_to_grid(bounds, params=p)
    df["LONCOL"], df["LATCOL"] = tbd.GPS_to_grid(df["lon"], df["lat"], params=p)
    score = df.groupby(["LONCOL", "LATCOL"])["total_score"].sum().reset_index()
    score.columns = ["LONCOL", "LATCOL", "total_score"]
    grid = pd.merge(grid, score, on=["LONCOL", "LATCOL"], how="left")
    return grid[grid["total_score"].notna()]

def classify_grid(grid):
    grid.sort_values(by="total_score", ascending=False, inplace=True)
    grid["num"] = pd.cut(grid["total_score"], bins=7, labels=["0", "1", "2", "3", "4", "5", "6"])
    return grid

def select_best_sites(df, grid):
    select = pd.merge(df[["geometry", "LONCOL", "LATCOL", "total_score"]],
                      grid[["LONCOL", "LATCOL", "num"]])
    select["num"] = select["num"].astype(int)
    select = select.sort_values(by=["LONCOL", "LATCOL", "total_score"], ascending=[True, True, False])
    f = select.groupby(["LONCOL", "LATCOL"]).apply(lambda x: x.head(x["num"].iloc[0])).reset_index(drop=True)
    return f

def main(filename, city, bounds, tbd):
    df_gz = read_data(filename, city)
    cols = df_gz.columns[4:]
    df_gz = calculate_scores(df_gz, cols)
    df_gz = extract_coordinates(df_gz)
    df_gz = create_geodataframe(df_gz)
    grid = score_grid(df_gz, bounds, tbd)
    grid = classify_grid(grid)
    best_sites = select_best_sites(df_gz, grid)
    return best_sites

# 调用主函数
# best_sites = main(filename, select_city, bounds, tbd)
